## Minnesota State COVID Response Analysis
This notebook contains the work to identify associations between the Minnesota state governmental response and the COVID-19 case count throughout the pandemic.


## Data Cleanup
As with most data mining projects, we will need to clean up the given data file in order to focus on the goal at hand. The "all-states-history.csv" file is a dataset of U.S. COVID-19 cases and deaths dating from the start of the pandemic to 11/29/20 and was sourced from [The Covid Tracking Project](https://covidtracking.com/data). We are analyzing 3 periods throughout this timeline:

- Early Breakout (Early March -> May)
- Summer (June -> August)
- Fall/Present (September -> Late November)

We will divide up the data into 3 different frames according to these periods.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import squarify
import seaborn as sns

In [ ]:
data = pd.read_csv('all-states-history.csv')

Cleaning up data to only include Minnesota instances and the appropriate attributes

In [ ]:
#isolating the columns we need
columns_to_show = ['date','state','death','deathConfirmed','deathIncrease','hospitalized','hospitalizedIncrease','negative'
                   ,'negativeIncrease','positive','positiveIncrease','totalTestResults','totalTestResultsIncrease']

#isolating only for MN data and putting in order March->November
clean_data = data[data['state'] == 'MN']
clean_data = clean_data[columns_to_show]
clean_data = clean_data.iloc[::-1]

#reindexing for weekly processing 
clean_data['date'] = clean_data['date'].astype('datetime64[ns]')
clean_data = clean_data.set_index('date')

# isolating the columns that need to be summed when converting to weekly index
columns_to_sum = clean_data[['deathIncrease','hospitalizedIncrease','negativeIncrease','positiveIncrease','totalTestResultsIncrease']]
weekly_data = columns_to_sum.resample('W', label='right', closed='right').sum()
weekly_data = weekly_data.reset_index()


# converting remaining non-sum columns to weekly index
remaining_cols = clean_data[['state','death','deathConfirmed','hospitalized', 'negative','positive','totalTestResults']]
remaining_cols = remaining_cols.resample('W').backfill().reset_index()
remaining_cols.head(39)

#merging and resetting the datframe order to be more clear
clean_data = pd.merge(remaining_cols, weekly_data, on='date').fillna(0)
clean_data = clean_data[['date','state','death','deathIncrease','deathConfirmed','hospitalized', 'hospitalizedIncrease','negative',
                        'negativeIncrease','positive', 'positiveIncrease','totalTestResults','totalTestResultsIncrease']]

In [ ]:
## Breaking down clean data into each period (earliest days at bottom of dataset)

early_breakout_data = clean_data[0:13]

summer_data = clean_data[13:26]

fall_data = clean_data[26:]

early_breakout_data.head(13)

bins = pd.cut(early_breakout_data['positiveIncrease'],4)

print(bins.shape)


## Analysis

Important MN Stats:

- Population (mn.gov estimate): 5,680,337
- Land Area (estimate): 79,610.08 sq. mi.
- Population Density: 71.35 people/sq. mi.




Early Breakout Period: